---

## Data Analysis

- This file differs from [2_data_analysis_1_base_data.ipynb](2_data_analysis_1_base_data.ipynb) in that it:
    - scales the base cleaned data created in [1_data_cleaning.ipynb](1_data_cleaning.ipynb).

Source dataset: 247076 rows × 37 columns
Processed and analyzed dataset: 247076 rows × 37 columns

---

In [1]:
# package imports go here
import pandas as pd
import numpy as np
import fastparquet as fp
import os
import sys
import pickle
import matplotlib.pyplot as plt
import importlib
import config
import time

sys.path.insert(1, config.package_path)
import ml_analysis as mlanlys
import ml_clean_feature as mlclean

In [2]:
start_time = time.time()

---

## 1. Read the cleaned dataset from file

---

In [3]:
# reload any changes to Config Settings
importlib.reload(config)

# BE SURE TO UPDATE THE LABEL FOR THIS ANALYSIS
# #############################
dataset_label = '3 Binary Dataset'
# #############################

year                        = config.year

clean_file                  = config.clean_file
performance_report          = config.performance_report

report_path                 = config.report_path
file_label                  = dataset_label.lower().replace(' ','_')
detailed_performance_report = report_path + file_label + '_detailed_performance_report.txt'

print(f"Year:                        {year}")
print(f"Clean File:                  {clean_file}")
print(f"Performance Report:          {performance_report}")
print(f"Detailed Performance Report: {detailed_performance_report}")

Year:                        2015
Clean File:                  data/brfss_2015_clean.parquet.gzip
Performance Report:          reports/performance_report.pkl
Detailed Performance Report: reports/3_binary_dataset_detailed_performance_report.txt


In [4]:
# Read final cleaned dataset from parquet file
df = pd.read_parquet(clean_file, engine="fastparquet")

In [5]:
diabetes_labels = df.columns

In [6]:
df.shape

(253680, 22)

---

## 2. Prepare the dataset for analysis

- Split the dataset into features and labels.
- Split the dataset into training and testing sets.
- Scale the dataset

---

In [7]:
from sklearn.datasets import make_regression, make_swiss_roll
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [8]:
# reload any changes to mlanlys
importlib.reload(mlanlys)

target = 'diabetes'
# Dictionary defining modification to be made to the base dataset
operation_dict = {  'target_column'     :  target,
                    'convert_to_binary' :  True,
                    'scaler'            : 'standard', # options: none, standard, minmax
                    'random_sample'     : 'none'      # options: none, undersample, oversample
                    }

# This insures that df if not modified during the call to modify_base_dataset()
df_modified = df.copy()

# Modify the base dataset
# data is returned where: X_train, X_test, y_train, y_test = data
data = mlanlys.modify_base_dataset(df_modified, operation_dict)

Base Dataset Modifications in Process
-------------------------------------
**Operation:target_column  diabetes
**Operation:convert_to_binary  True
  -- Converting dataset to binary (0,1) from (0,1,2)


****Cleaning Feature: diabetes
  Initial Unique features in [diabetes]:  [0. 1. 2.]
  values_to_drop: ********* NO Parameters were specified *********
  translate: {1: 0, 2: 1}
  scale: ********* NO Parameters were specified *********
  FINAL Unique features in [diabetes]:  [0. 1.]
**Operation:scaler  standard
  -- Performing train_test_split on dataframe with target:'diabetes'
     -- Run automatically before scalar or random_sample operations
  -- Performing StandardScaler on X_train: Updates X_train, y_test
**Operation:random_sample  none

Dataframe, Train Test Summary
-----------------------------
Dataframe: (253680, 22)  Data:4, X_train:190260, y_train:190260, X_test:63420, y_test:63420
ValueCounts:   y_train: len:2   0: 163660   1: 26600
ValueCounts:   y_test : len:2   0:  54674  

In [9]:
# Print some statistics about the original df and the modified dataframe
print(f"Original Dataframe")
print(f"------------------")
print(f"df.shape: {df.shape}")
print(f"df[{target}].value_counts:  {df[target].value_counts()}")

print(f"\nModified Dataframe")
print(f"------------------")
print(f"df_modified.shape: {df_modified.shape}")
print(f"df_modified[{target}].value_counts:  {df_modified[target].value_counts()}")

Original Dataframe
------------------
df.shape: (253680, 22)
df[diabetes].value_counts:  diabetes
0.0    213703
2.0     35346
1.0      4631
Name: count, dtype: int64

Modified Dataframe
------------------
df_modified.shape: (253680, 22)
df_modified[diabetes].value_counts:  diabetes
0.0    218334
1.0     35346
Name: count, dtype: int64


In [10]:
X_train, X_test, y_train, y_test = data
print(f"Dataframe: {df_modified.shape}  Data:{len(data)}, X_train:{len(X_train)}, y_train:{len(y_train)}, X_test:{len(X_test)}, y_test:{len(y_test)}")
y_train.value_counts()

Dataframe: (253680, 22)  Data:4, X_train:190260, y_train:190260, X_test:63420, y_test:63420


diabetes
0.0    163660
1.0     26600
Name: count, dtype: int64

In [11]:
y_test.value_counts()

diabetes
0.0    54674
1.0     8746
Name: count, dtype: int64

---

## 3. Run initial Tests and get k_value

**From step 2:**  Data = [X_train_modified, X_test_modified, y_train_modified, y_test]

---

In [12]:
# reload any changes to mlanlys
importlib.reload(mlanlys)

# Determine the k_value
# mlanlys.knn_plot(data)

<module 'ml_analysis' from '/mnt/c/ML/DU/repos/projects/project-2/DU-project-2-2015/brfss_2015/../pkgs/ml_analysis.py'>

**Note:** From the knn plot above, pick a k-value of 3.

---

## 4. Run the Analysis

---

#### Model Run Times

-  Base dataset (247076 rows × 37 columns):

| Model | Run Time |
| ----- | -------- |
| test_model(SVC(kernel='linear'), data)                          | Aborted >35min (Data too large, consider for RandomUndersampling dataset) |
| test_model(KNeighborsClassifier(n_neighbors=k_value), data)     | 247.13 seconds |
| test_model(tree.DecisionTreeClassifier(), data)                 |   3.89 seconds |
| test_model(RandomForestClassifier(), data)                      |  60.94 seconds |
| test_model(ExtraTreesClassifier(random_state=1), data)          |  58.54 seconds |
| test_model(GradientBoostingClassifier(random_state=1), data)    | 115.21 seconds |
| test_model(AdaBoostClassifier(random_state=1), data)            |  11.91 seconds |
| test_model(LogisticRegression(), data)                          |   4.90 seconds |
| **Total** w/o SVC| 502.52 seconds / **8:23 minutes** |

In [13]:
# reload any changes to nlanlys
importlib.reload(mlanlys)

k_value = 3

#### COMMENT OUT ONE OF THE FOLLOWING SECTIONS

## SECTION 1
# Capture stdout & stderr into two strings: osc.stdout and osc.stderr that contain the output from the function
# -- This allows the output to be printed here or to a file or both.

with mlanlys.OutStreamCapture() as osc:
    performance_summary = mlanlys.run_classification_models(data, k_value)
#    performance_summary = mlanlys.run_classification_models_test(data, k_value)

## <OR>
## SECTION 2

# performance_summary = mlanlys.run_classification_models(data, k_value)


In [14]:
# UNCOMMENT if using SECTION 1 in the previous step
# print(osc.stdout)

# Add code to print osc.stdout to a file if desired.

---

### 4.1 Archive Performance Summary

- For use in Project-2 Performance Summary Report
---

In [15]:
# performance_summary is a dataframe of performance statistics

# Add the dataset label as the first column in performance_summary
dataset_column = pd.Series([dataset_label] * len(performance_summary), name=dataset_label)
performance_summary.insert(0, 'new_column', dataset_column)

analysis_perf_summary = { 'dataset_size': list(df.shape), 'report': performance_summary}

# Performance_report is a file containing all the performance summary statistics
if os.path.exists(performance_report):
    print(f"The file {performance_report} exists.")
    # Load Performance Report
    with open(performance_report, 'rb') as file: perf_report = pickle.load(file)
else:
    print(f"The file {performance_report} does not exist.")
    perf_report = {}
    
perf_report[dataset_label] = analysis_perf_summary

# Save Performance Report
with open(performance_report, 'wb') as file: pickle.dump(perf_report, file)

The file reports/performance_report.pkl exists.


### 4.2 Archive the Performance Detailed Statistics Report
---

In [16]:
# osc.stdout contains the details of the performance statistics

with open(detailed_performance_report, "w") as file:
    file.write(osc.stdout)

---

## 5. Performance Summary

---

In [17]:
# print the performance summary
print(f"******************************************")
print(f"Performance Summary for: {dataset_label}")
print(f"******************************************")

performance_summary

******************************************
Performance Summary for: 3 Binary Dataset
******************************************


,new_column,model,slice,score,balanced_accuracy,roc_auc_score,Mean Squared Error,Accuracy,Precision,Recall,F1-score,Specificity,False Positive Rate,Matthews Correlation Coefficient
0,3 Binary Dataset,KNeighborsClassifier,Train,0.9024,0.7268,0.9340,0.0976,0.9024,0.7276,0.4831,0.5807,0.9706,0.0294,NaN
1,3 Binary Dataset,KNeighborsClassifier,Test,0.8371,0.5851,0.6772,0.1629,0.8371,0.3619,0.2370,0.2864,0.9331,0.0669,0.2047
2,3 Binary Dataset,DecisionTreeClassifier,Train,0.9944,0.9805,0.9998,0.0056,0.9944,0.9991,0.9611,0.9797,0.9999,0.0001,NaN
3,3 Binary Dataset,DecisionTreeClassifier,Test,0.7958,0.5959,0.5960,0.2042,0.7958,0.2856,0.3199,0.3018,0.8720,0.1280,0.1831
4,3 Binary Dataset,RandomForestClassifier,Train,0.9944,0.9819,0.9994,0.0056,0.9944,0.9953,0.9645,0.9797,0.9993,0.0007,41.2378
5,3 Binary Dataset,RandomForestClassifier,Test,0.8600,0.5712,0.7990,0.1400,0.8600,0.4786,0.1725,0.2536,0.9699,0.0301,0.2260
6,3 Binary Dataset,ExtraTreesClassifier,Train,0.9944,0.9805,0.9998,0.0056,0.9944,0.9991,0.9611,0.9797,0.9999,0.0001,NaN
7,3 Binary Dataset,ExtraTreesClassifier,Test,0.8526,0.5686,0.7778,0.1474,0.8526,0.4180,0.1764,0.2481,0.9607,0.0393,0.2020
8,3 Binary Dataset,GradientBoostingClassifier,Train,0.8668,0.5734,0.8319,0.1332,0.8668,0.5836,0.1661,0.2586,0.9807,0.0193,0.2605
9,3 Binary Dataset,GradientBoostingClassifier,Test,0.8670,0.5707,0.8301,0.1330,0.8670,0.5616,0.1617,0.2511,0.9798,0.0202,0.2498


In [18]:
print(f"Completed: Execution Time %s seconds:" % round((time.time() - start_time),2) )

Completed: Execution Time 383.66 seconds:


---

## 6. Conclusions

- A first glance at the summary, it appears that the Boosting models may have performed well with test/train scores were >.8 and similar in scale (<.02 delta).  However, the poor test confusion matrix and balanced accuracy highlight the overfitting.

- The Base Cleaned data is overfit as indicated by:
    - Poor confusion matrix on the detailed report for test sets on all models
    - Low balanced accuracy as compared to the model score (less than 50%)


---

---